## Import Packages

In [1]:
import pandas as pd
import numpy as np
import os
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
from tqdm.notebook import tqdm
import tqdm.notebook as tq
import cv2

## Read Data

In [2]:
%cd "/kaggle/input"
train = pd.read_csv("../input/smartathon/dataset/train.csv")
test = pd.read_csv("../input/smartathon/dataset/test.csv")
submission = pd.read_csv("../input/smartathon/dataset/sample_submission.csv")

/kaggle/input


## Manually Evaluated Data

#### We have manually evaluated data and removed objects with wrong labels and image with multiple bounding boxes as below:

## Wrong Object

### Example 1
<a href="https://ibb.co/2M90rn2"><img src="https://i.ibb.co/5j0X3MD/13cf1579b72ca53bed371cf9a6d236a2.jpg" alt="13cf1579b72ca53bed371cf9a6d236a2" border="0"></a>

### Example 2
<a href="https://ibb.co/N3X3Lf4"><img src="https://i.ibb.co/yRDR8K3/1883acc049017ac936ef14ddeef51f21.jpg" alt="1883acc049017ac936ef14ddeef51f21" border="0"></a>

## Multiple Objects

### Example 1
<a href="https://ibb.co/rF9Qmyr"><img src="https://i.ibb.co/2WJFyK4/02533dafc548f483ce858ce385dfba6b.jpg" alt="02533dafc548f483ce858ce385dfba6b" border="0"></a>

### Example 2
<a href="https://ibb.co/Nn31KY0"><img src="https://i.ibb.co/YTWDhj5/03858da6a532e19e7fdbe35530bf2447.jpg" alt="03858da6a532e19e7fdbe35530bf2447" border="0"></a>

In [3]:
images_to_delete = ["00f6cb1c7473f12ea482dceaad648470.jpg","0422d536e5855168fa4d2f8fda42a88e.jpg","2acbba0d2b1a3b294b4aa381afe1f9ca.jpg","028807524edee10c130e14605688a69e.jpg","053d4e9e51ac3a2c944f2c252a089754.jpg","070ff13c834ea8c4c3624ef734f84f5c.jpg","0bc4467f2ceacceecb211c73c6f33d08.jpg","02533dafc548f483ce858ce385dfba6b.jpg",
"03858da6a532e19e7fdbe35530bf2447.jpg",
"0450fc05a731b77d11766a2cd4ada1b0.jpg",
"0543a0c0b6fee43ec27d99cccc54961a.jpg",
"05b9d238c199dd6c831e2072636c952b.jpg",
"068b3c2a02ee7cbc3e0b293664985a7b.jpg",
"0a2bc0dc2371794509f4b776aff0dd88.jpg",
"0e506ddbe9e654f92c6cd97bf9899ba6.jpg",
"0ee43fc7ba28742044104c23e7abafd7.jpg",
"1299e5c065f4f5ce827161a4b754b6e4.jpg",
"1840a781d3437c617b91c960c92cb7ea.jpg",
"252550a0e4396724c8ba3d4a81e7ce97.jpg",
"262c7a0daffc4a0548b4cefce8b9651a.jpg",
"291d3a6d38674da0e4b05135008bceb1.jpg",
                   "13cf1579b72ca53bed371cf9a6d236a2.jpg",
                   "1883acc049017ac936ef14ddeef51f21.jpg",
                   "217a94383020937a565f6caa58c40525.jpg"]

In [4]:
for i,j in enumerate(train['image_path']):
    if j in images_to_delete:
        train.drop(i,inplace=True)
train = train.reset_index(drop=True)

## If we multiply bounding box by 2, its creating a larger area of bounding box that might be a wrong training dataset.
## Since final width of the image is 960, reducing size of the image by 50%

In [ ]:
# %cd /kaggle/input
# for i,j in enumerate(test['image_path']):
#     img = cv2.imread("../input/smartathon/dataset/images/" + j.split("/")[-1], cv2.IMREAD_UNCHANGED) 
#     width = 960
#     height = 540 # keep original height
#     dim = (width, height)
#     # resize image
#     resized = cv2.resize(img, dim)
#     cv2.imwrite("/kaggle/working/smartathon/resize_images/" + j.split("/")[-1],resized)

In [5]:
## Above code block was used to reduce size of the image, we have created 
## a notebook to reuse those images instead of running it again and again -- Refer resize images dataset
os.makedirs('/kaggle/working/smartathon/resize_images', exist_ok = True)
## TO avoid repetation resized image is created using above code
path_resize_image = "/kaggle/input/resize-images/smartathon/resize_images/"
for i,j in enumerate(train['image_path']):
    shutil.copy(path_resize_image + str(j) , '/kaggle/working/smartathon/resize_images')

## Plot Bounding Box images

In [6]:
import cv2

def plot_one_box(x, img, color=None, label=None, line_thickness=None, Inverted=False):
    # Plots one bounding box on image img
    tl = line_thickness or 2 # line thickness
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl)
    if label:
        tf = tl # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 2, thickness=tf)[0]
    if Inverted == True:
        c1 = c2
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
    else:
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1) # filled
        cv2.putText(
        img,
        label,
        (c1[0], c1[1] - 2),
        0,
        tl / 2,
        [225, 255, 255],
        thickness=tf,
        lineType=cv2.LINE_AA,
        )

In [7]:
# # Using readlines()
# file1 = open('../input/smartathon/dataset/train.csv', 'r')
# Lines = file1.readlines()

# count = 0
# # Strips the newline character
# for line in Lines:
#     #print(line)
#     if count == 0:
#         count += 1
#         continue
# print(count)
# file_id_path = line.split(',')[1]
# print(file_id_path)
# # open image in cv2
# img = cv2.imread("../input/smartathon/dataset/images/" + file_id_path)
# #print(img)
# h, w, c = img.shape
# cat = line.split(',')[2]
# xmax = int(float(line.split(',')[3])) * 2
# xmin = int(float(line.split(',')[4])) * 2
# ymax = int(float(line.split(',')[5])) * 2
# ymin = int(float(line.split(',')[6])) * 2
# # plot the box
# plot_one_box([xmin, ymin, xmax, ymax], img, color=(0, 255, 0), label=cat, line_thickness=2)
# # save the image
# # you might need to create the folder "drawn" first!
# cv2.imwrite("../working/" + file_id_path, img)
# print("Line {}: {}".format(count, line.strip()))
# count += 1

In [8]:
## Keeping contant height and width
train['height'] = 540
train['width'] = 960

## Fixing issues with bounding boxes

* ### Keeping Xmax to maximum size of 960
* ### Keeping Xmin to zero if its negative
* ### Keeping Ymax to maximum size of 540
* ### Keeping Ymin to zero if its negative

In [9]:
### fixing bounding box problems

for i,j in enumerate(train['xmax']):
    ##### xmax ---> 0 <= xmax <= 960
    if  j > 960:
        train['xmax'].iloc[i] = 960
    elif j < 0:
        train['xmax'].iloc[i] = 960
    
    #### xmin ----> 0 <= xmin <= xmax
    
    if train['xmin'].iloc[i] < 0 :
        train['xmin'].iloc[i] =0
    elif train['xmin'].iloc[i] > train['xmax'].iloc[i]:
        print("####### highlighting index xmin <= xmax ####" + str(i))
    
    ######## ymax ---> 0<= yamx <= 540
    
    if train['ymax'].iloc[i] > 540 :
        train['ymax'].iloc[i] =540
    elif train['ymax'].iloc[i] < 0:
        train['ymax'].iloc[i] = 540
    
    ######## ymin ---> 0<= ymin <= ymax
    if train['ymin'].iloc[i] < 0 :
        train['ymin'].iloc[i] =0
    elif train['ymin'].iloc[i] > train['ymax'].iloc[i]:
        print("####### highlighting index ymin <= ymax ####" + str(i))

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
train = train.rename(columns={"xmax":"x_max","xmin":"x_min","ymax":"y_max","ymin":"y_min","class":"class_id","name":"class_name"})

## Normalizing Input of the image

In [11]:
train['x_min'] = train.apply(lambda row: (row.x_min)/row.width, axis =1)
train['y_min'] = train.apply(lambda row: (row.y_min)/row.height, axis =1)

train['x_max'] = train.apply(lambda row: (row.x_max)/row.width, axis =1)
train['y_max'] = train.apply(lambda row: (row.y_max)/row.height, axis =1)

train['x_mid'] = train.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train['y_mid'] = train.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train['w'] = train.apply(lambda row: (row.x_max-row.x_min), axis =1)
train['h'] = train.apply(lambda row: (row.y_max-row.y_min), axis =1)

train['area'] = train['w']*train['h']
train.head()

,class_id,image_path,class_name,x_max,x_min,y_max,y_min,height,width,x_mid,y_mid,w,h,area
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,0.830208,0.730208,0.485185,0.390741,540,960,0.780208,0.437963,0.100000,0.094444,0.009444
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,0.970833,0.818750,0.609259,0.440741,540,960,0.894792,0.525000,0.152083,0.168519,0.025629
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,0.766667,0.684375,0.509259,0.424074,540,960,0.725521,0.466667,0.082292,0.085185,0.007010
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,BAD_BILLBOARD,1.000000,0.818750,0.251852,0.000000,540,960,0.909375,0.125926,0.181250,0.251852,0.045648
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,SAND_ON_ROAD,0.694792,0.571875,0.422222,0.331481,540,960,0.633333,0.376852,0.122917,0.090741,0.011154


In [12]:
class_ids, class_names = list(zip(*set(zip(train.class_id, train.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

['GRAFFITI',
 'FADED_SIGNAGE',
 'POTHOLES',
 'GARBAGE',
 'CONSTRUCTION_ROAD',
 'BROKEN_SIGNAGE',
 'BAD_STREETLIGHT',
 'BAD_BILLBOARD',
 'SAND_ON_ROAD',
 'CLUTTER_SIDEWALK',
 'UNKEPT_FACADE']

In [13]:
### Create path for label text files
os.makedirs('/kaggle/working/smartathon/label_text_file', exist_ok = True)

In [14]:
train['bbox']= train[['class_id','x_mid','y_mid','w','h']].values.tolist()

In [15]:
train.head()

,class_id,image_path,class_name,x_max,x_min,y_max,y_min,height,width,x_mid,y_mid,w,h,area,bbox
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,0.830208,0.730208,0.485185,0.390741,540,960,0.780208,0.437963,0.100000,0.094444,0.009444,"[3.0, 0.7802083333333334, 0.43796296296296294,..."
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,0.970833,0.818750,0.609259,0.440741,540,960,0.894792,0.525000,0.152083,0.168519,0.025629,"[3.0, 0.8947916666666667, 0.525, 0.15208333333..."
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,0.766667,0.684375,0.509259,0.424074,540,960,0.725521,0.466667,0.082292,0.085185,0.007010,"[3.0, 0.7255208333333334, 0.4666666666666667, ..."
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,BAD_BILLBOARD,1.000000,0.818750,0.251852,0.000000,540,960,0.909375,0.125926,0.181250,0.251852,0.045648,"[7.0, 0.909375, 0.1259259259259259, 0.18125000..."
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,SAND_ON_ROAD,0.694792,0.571875,0.422222,0.331481,540,960,0.633333,0.376852,0.122917,0.090741,0.011154,"[8.0, 0.6333333333333333, 0.3768518518518519, ..."


In [16]:
new_df = train.groupby("image_path")['bbox'].apply(list).reset_index(name="bbox")

In [17]:
new_df.head()

,image_path,bbox
0,000b0fd767c5ab2656b7ed8d43674777.jpg,"[[3.0, 0.46562499999999996, 0.5324074074074074..."
1,000ed1547634a24f09f22530065d46c9.jpg,"[[7.0, 0.6270833333333333, 0.32685185185185184..."
2,0011b4d3f9068a9efbb84dc3157b76c9.jpg,"[[3.0, 0.9421875, 0.4703703703703704, 0.115624..."
3,0019a9ab6fd5ef3df915241bc323afb5.jpg,"[[4.0, 0.2552083333333333, 0.5861111111111111,..."
4,00349ffee00a2ce026ffb36e4e37caab.jpg,"[[3.0, 0.840625, 0.38888888888888884, 0.064583..."


## Creating text label files for YOLO training

In [18]:
def process_images(df):
    for _, row in tq.tqdm(df.iterrows(), total=len(df)):
        yolo_data = row['bbox']
        yolo_data = np.array(yolo_data)
        image_id = row['image_path'].split('.')[0]
        np.savetxt(os.path.join('yolov5/base_dir', 
                    f"/kaggle/working/smartathon/label_text_file/{image_id}.txt"),
                    yolo_data, 
                    fmt=["%d", "%f", "%f", "%f", "%f"]
                    ) # fmt means format the columns

In [19]:
process_images(new_df)

  0%|          | 0/7850 [00:00<?, ?it/s]

## Splitting the data for training and validation

In [20]:
new_df.head()

,image_path,bbox
0,000b0fd767c5ab2656b7ed8d43674777.jpg,"[[3.0, 0.46562499999999996, 0.5324074074074074..."
1,000ed1547634a24f09f22530065d46c9.jpg,"[[7.0, 0.6270833333333333, 0.32685185185185184..."
2,0011b4d3f9068a9efbb84dc3157b76c9.jpg,"[[3.0, 0.9421875, 0.4703703703703704, 0.115624..."
3,0019a9ab6fd5ef3df915241bc323afb5.jpg,"[[4.0, 0.2552083333333333, 0.5861111111111111,..."
4,00349ffee00a2ce026ffb36e4e37caab.jpg,"[[3.0, 0.840625, 0.38888888888888884, 0.064583..."


In [21]:
new_df['image_path'] = "/kaggle/input/smartathon/dataset/images/" + new_df['image_path']

In [22]:
from sklearn.model_selection import train_test_split
train_files, val_files = train_test_split(new_df, test_size=0.2, random_state=101)

In [23]:
train_image_path = train_files['image_path']
validation_image_path = val_files['image_path']

## Copying Files to create YOLO structure 

In [24]:
os.makedirs('/kaggle/working/smartathon/labels/train', exist_ok = True)
os.makedirs('/kaggle/working/smartathon/labels/val', exist_ok = True)
os.makedirs('/kaggle/working/smartathon/images/train', exist_ok = True)
os.makedirs('/kaggle/working/smartathon/images/val', exist_ok = True)
label_dir = '/kaggle/working/smartathon/label_text_file'
for file in tqdm(train_image_path):
    shutil.copy(file, '/kaggle/working/smartathon/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/smartathon/labels/train')
    
for file in tqdm(validation_image_path):
    shutil.copy(file, '/kaggle/working/smartathon/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/smartathon/labels/val')

  0%|          | 0/6280 [00:00<?, ?it/s]

  0%|          | 0/1570 [00:00<?, ?it/s]

In [25]:
!pip install seedir
import seedir as sd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 1.8 MB/s eta 0:00:00a 0:00:01


In [26]:
sd.seedir('/kaggle/working/smartathon',itemlimit=6, style='emoji')

📁 smartathon/
├─📁 images/
│ ├─📁 val/
│ │ ├─📄 3bff5c10910a6ca1478303106e966cb3.jpg
│ │ ├─📄 9092e30ca3094302644a330ae782fb65.jpg
│ │ ├─📄 3ff87a4ecbc2c06e588917e78a212df4.jpg
│ │ ├─📄 1568df8de13520b917fd1313c37a83f7.jpg
│ │ ├─📄 83fc281880965a8dad9715fb49347bde.jpg
│ │ └─📄 46d80b27ae874ea2c1728e74916bdb91.jpg
│ └─📁 train/
│   ├─📄 92b78794d995882012fdb6afdeb882db.jpg
│   ├─📄 13ec33f0dc03bfc5425a0d3e0e4994bf.jpg
│   ├─📄 f643dbf2b0a3d3bae9c90b3eac754cd3.jpg
│   ├─📄 fde48bcb170bfa2134901bdc7e6ba81c.jpg
│   ├─📄 ed7366abcf76044aca59f54f377720c1.jpg
│   └─📄 5b47296c7786bc73c0f551b8d2c4197d.jpg
├─📁 labels/
│ ├─📁 val/
│ │ ├─📄 7126e8674ecf638a9586ce5ca9b91f8d.txt
│ │ ├─📄 f27af32d593cd546ad11e518b7c28a94.txt
│ │ ├─📄 83d142b0d50c210abdbfef35a8024136.txt
│ │ ├─📄 fe2979002129e876d31957d970029c78.txt
│ │ ├─📄 297574b7bf8aaf42eb25af7f8a3e95be.txt
│ │ └─📄 fc410c3be2a222d0de857789fed6b5a8.txt
│ └─📁 train/
│   ├─📄 60785578f9a4976481a0a3ce27ce3eda.txt
│   ├─📄 cfde192f90708ace990df2abec57ade7.txt
│   ├─📄 bf045a

## YOLOv5 Required files for training


In [27]:
len(classes)

11

In [28]:
from os import listdir
from os.path import isfile, join
import yaml

cwd = '/kaggle/working/'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('/kaggle/working/smartathon/images/train/*'):
        f.write(path+'\n')
            
with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('/kaggle/working/smartathon/images/val/*'):
        f.write(path+'\n')

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    nc    = 11,
    names = classes
    )

with open(join( cwd , 'smartathon.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'smartathon.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- GRAFFITI
- FADED_SIGNAGE
- POTHOLES
- GARBAGE
- CONSTRUCTION_ROAD
- BROKEN_SIGNAGE
- BAD_STREETLIGHT
- BAD_BILLBOARD
- SAND_ON_ROAD
- CLUTTER_SIDEWALK
- UNKEPT_FACADE
nc: 11
train: /kaggle/working/train.txt
val: /kaggle/working/val.txt



In [29]:
# https://www.kaggle.com/ultralytics/yolov5
# !git clone https://github.com/ultralytics/yolov5  # clone repo
# %cd yolov5
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')
# %pip install -qr requirements.txt # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.11.0 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [30]:
!pip install wandb

In [31]:
!wandb login "" ## Add wandb key

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Modeling with Yolov5

In [32]:
## Hyper parameter tunning allows to train for longer duration without overfitting

hyp = dict( lr0= 0.01,  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf= 0.1,  # final OneCycleLR learning rate (lr0 * lrf)
momentum= 0.937,  # SGD momentum/Adam beta1
weight_decay= 0.0005,  # optimizer weight decay 5e-4
warmup_epochs= 3.0,  # warmup epochs (fractions ok)
warmup_momentum= 0.8,  # warmup initial momentum
warmup_bias_lr= 0.1,  # warmup initial bias lr
box= 0.05,  # box loss gain
cls= 0.3,  # cls loss gain
cls_pw= 1.0,  # cls BCELoss positive_weight
obj= 0.7,  # obj loss gain (scale with pixels)
obj_pw= 1.0,  # obj BCELoss positive_weight
iou_t= 0.20,  # IoU training threshold
anchor_t= 4.0,  # anchor-multiple threshold
# anchors= 3,  # anchors per output layer (0 to ignore)
fl_gamma= 0.0,  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h= 0.015,  # image HSV-Hue augmentation (fraction)
hsv_s= 0.7,  # image HSV-Saturation augmentation (fraction)
hsv_v= 0.4,  # image HSV-Value augmentation (fraction)
degrees= 0.0,  # image rotation (+/- deg)
translate= 0.1,  # image translation (+/- fraction)
scale= 0.9,  # image scale (+/- gain)
shear= 0.0,  # image shear (+/- deg)
perspective= 0.0,  # image perspective (+/- fraction), range 0-0.001
flipud= 0.0,  # image flip up-down (probability)
fliplr= 0.5,  # image flip left-right (probability)
mosaic= 1.0,  # image mosaic (probability)
mixup= 0.1,  # image mixup (probability)
copy_paste= 0.1  # segment copy-paste (probability)
          )

with open(join( cwd , 'hyp.yaml'), 'w') as outfile:
    yaml.dump(hyp, outfile, default_flow_style=False)

f = open(join( cwd , 'hyp.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
anchor_t: 4.0
box: 0.05
cls: 0.3
cls_pw: 1.0
copy_paste: 0.1
degrees: 0.0
fl_gamma: 0.0
fliplr: 0.5
flipud: 0.0
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
iou_t: 0.2
lr0: 0.01
lrf: 0.1
mixup: 0.1
momentum: 0.937
mosaic: 1.0
obj: 0.7
obj_pw: 1.0
perspective: 0.0
scale: 0.9
shear: 0.0
translate: 0.1
warmup_bias_lr: 0.1
warmup_epochs: 3.0
warmup_momentum: 0.8
weight_decay: 0.0005



In [34]:
%cd "/kaggle/working/yolov5/"
!WANDB_MODE="online" python train.py --img 688 --batch 16 --epochs 35 --data /kaggle/working/smartathon.yaml --hyp /kaggle/working/hyp.yaml  --weights yolov5x.pt --cache

/kaggle/input
wandb: Currently logged in as: buntyshah. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.13.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.21
wandb: Run data is saved locally in /kaggle/working/yolov5/wandb/run-20230121_092223-31oth31h
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run exp2
wandb: ⭐️ View project at https://wandb.ai/buntyshah/YOLOv5
wandb: 🚀 View run at https://wandb.ai/buntyshah/YOLOv5/runs/31oth31h
Scanning '/kaggle/working/smartathon/labels/train.cache' for images and labels..
Caching images (5.3GB): 100%|███████████████| 6280/6280 [02:26<00:00, 42.89it/s]
Scanning '/kaggle/working/smartathon/labels/val.cache' for images and labels... 
Caching images (1.3GB): 100%|███████████████| 1570/1570 [01:02<00:00, 25.04it/s]
Plotting labels... 

Analyzing anchors... anchors/target = 4.98, Best Possible Recall (BPR) = 0.9997
      0/34     8.73G   0.08

## Inference and preparing for final submission

In [38]:
%cd "/kaggle/input"

/kaggle/input


In [39]:
os.makedirs('/kaggle/working/smartathon/test', exist_ok = True)

In [42]:
%cd /kaggle/working/yolov5
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/kaggle/working/yolov5/runs/train/exp2/weights/best.pt', force_reload=True) 

/kaggle/working/yolov5


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-1-21 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 484 layers, 88457904 parameters, 0 gradients
Adding AutoShape... 


In [44]:
# %cd /kaggle/input
# for i,j in enumerate(test['image_path']):
#     img = cv2.imread("../input/smartathon/dataset/images/" + j.split("/")[-1], cv2.IMREAD_UNCHANGED) 
#     width = 960
#     height = 540 # keep original height
#     dim = (width, height)
#     # resize image
#     resized = cv2.resize(img, dim)
#     cv2.imwrite("/kaggle/working/smartathon/resize_images/" + j.split("/")[-1],resized)

In [45]:
## Above code block was used to reduce size of the image, we have created 
## a notebook to reuse those images instead of running it again and again -- Refer resize images dataset
%cd /kaggle/input
## TO avoid repetation resized image is created using above code
path_resize_image = "/kaggle/input/resize-images/smartathon/resize_images/"
for i,j in enumerate(test['image_path']):
    shutil.copy(path_resize_image + str(j) , '/kaggle/working/smartathon/resize_images')

/kaggle/input


In [46]:
test['image_path'] = "/kaggle/working/smartathon/resize_images/" + test['image_path']

In [77]:
%cd "/kaggle/working"
final_output = pd.DataFrame(columns=["xmin", "ymin", "xmax", "ymax", "confidence", "class", "name", "image_path"])
for i,j in enumerate(test['image_path']):
    results = model(j,size=704,augment=True)
    output = results.pandas().xyxy[0]
    output['image_path'] = j.split("/")[-1]
    output = output[output['confidence'] >= 0.4]
    if output.shape[0] == 0:
        output = output.append(pd.DataFrame([[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,j.split("/")[-1]]],columns=["xmin", "ymin", "xmax", "ymax", "confidence", "class", "name", "image_path"]),ignore_index=True)
    final_output = final_output.append(output)

/kaggle/working


In [78]:
final_output = final_output.sort_values(['image_path','confidence'],ascending=False)

In [79]:
### change columnsput format
final_output = final_output.reset_index(drop=True)
final_output = final_output[["class","image_path","name","xmax","xmin","ymax","ymin","confidence"]]

In [80]:
final_output.to_csv("ThemeV2_700px_16bz_35_0.4_hyp_augT.csv",index=False)

In [ ]:
%cd "/kaggle/working"
# Image
img = test['image_path'].iloc[7]

# Inference
results = model(img)

results.pandas().xyxy[0].shape[0]

In [ ]:
results.show()

In [54]:
## Zip weights for further use
%cd /kaggle/working
!tar -zcvf weights.tar.gz /kaggle/working/yolov5/runs/train/exp2/weights/

/kaggle/working
tar: Removing leading `/' from member names
/kaggle/working/yolov5/runs/train/exp2/weights/
/kaggle/working/yolov5/runs/train/exp2/weights/best.pt
/kaggle/working/yolov5/runs/train/exp2/weights/last.pt
